In [ ]:
---
  title: "EX 06"
  author: "Maksim Kulik"
  format:
    html:
      embed-resources: true
  toc: true
  jupyter: python3
  ---

## Step 1: Setup and Data Preprocessing

- Start by importing the necessary libraries and load the spam.csv dataset.

- Preprocess the data by encoding categorical variables, defining features and target, and splitting the data into training and testing sets. Finally, apply PCA to reduce dimensionality.

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

sns.set(font_scale = 1.25)
sns.set_style("white")


### Data overview

In [2]:
data = pd.read_csv('data/spam.csv')
print(f'First 5 rows:\n{data.head()}', f'Data types:\n{data.dtypes}', f'The dataset contains {data.shape[0]} rows and {data.shape[1]} columns.', sep='\n\n')

First 5 rows:
   crl.tot  dollar   bang  money  n000  make yesno
0      278   0.000  0.778   0.00  0.00  0.00     y
1     1028   0.180  0.372   0.43  0.43  0.21     y
2     2259   0.184  0.276   0.06  1.16  0.06     y
3      191   0.000  0.137   0.00  0.00  0.00     y
4      191   0.000  0.135   0.00  0.00  0.00     y

Data types:
crl.tot      int64
dollar     float64
bang       float64
money      float64
n000       float64
make       float64
yesno       object
dtype: object

The dataset contains 4601 rows and 7 columns.


## Step 2: Model Training and Decision Boundary Visualization

- Train a Decision Tree classifier on the PCA-transformed training data.

- Implement and use the `decisionplot` function (from the lecture) to visualize the decision boundary of your trained model.

## Step 3: Model Evaluation

- Evaluate your model using accuracy, precision, recall, F1 score, and AUC-ROC metrics.

## Assignment:
- Implement the missing parts of the code: the decisionplot function and AUC-ROC calculation.

- Discuss the results among your peers. Consider the following:

    - Which metric is most informative for this problem and why?

    - How does the decision boundary visualization help in understanding the model’s performance?

    - Reflect on the impact of PCA on model performance and decision boundary.

